In [7]:
import h5py
%pylab
from meshoid import Meshoid
from scipy.fft import ifftn, fftn, fftfreq


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


/home/mike/.local/lib/python3.7/site-packages/IPython/core/magics/pylab.py:160: UserWarning: pylab import has clobbered these variables: ['fftn', 'ifftn']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [18]:
F = h5py.File("snapshot_025.hdf5",'r')
x = np.array(F["PartType0/Coordinates"])
m = np.array(F["PartType0/Masses"])
v = np.array(F["PartType0/Velocities"])
h = np.array(F["PartType0/SmoothingLength"])
density = np.array(F["PartType0/Density"])

M = Meshoid(x,m,h)#,boxsize=F["Header"].attrs["BoxSize"],n_jobs=1)
res = 125
#rho = M.DepositToGrid(m,res=res,size=F["Header"].attrs["BoxSize"], center=np.repeat(F["Header"].attrs["BoxSize"]/2,3))
#print(rho.min())
#rhov = np.array([M.DepositToGrid(m*v[:,i],res=res)/rho for i in range(3)])
#%timeit rho2 = M.InterpToGrid(density,res=res)

vgrid = np.array([M.InterpToGrid(v[:,i],res=res,size=F["Header"].attrs["BoxSize"], center=np.repeat(F["Header"].attrs["BoxSize"]/2,3)) for i in range(3)])
np.isnan(vgrid).sum()
# = np.array([M.DepositToGrid(x[:,i],res=res,method='nearest') for i in range(3)])
#rho = M.DepositToGrid(np.array(neF["PartType0"]["Density"]),res=res,method='kernel')

0

In [19]:
plt.imshow(vgrid[2][:,:,0])

In [ ]:
%pylab
from scipy import interpolate

N = 32
x1 = np.linspace(0.5/N, 1-0.5/N, N) # original coords                                                                    
xx,yy = np.meshgrid(x1,x1)
x2 = np.linspace(0.25/N,1-0.25/N, 2*N) # new coords   
#xx, yy = np.meshgrid(x1, x1)
z = np.sin(xx**2+yy**2)

#f = interpolate.interp2d(x, y, z, kind='cubic')

import matplotlib.pyplot as plt

#xnew = np.arange(-5.0, 5.05, 1e-2)

#ynew = np.arange(-5.0, 5.05, 1e-2)

znew = interpolate.RectBivariateSpline(x1, x1, z)(x2, x2)
plt.imshow(znew)
#plt.plot(x, z[0, :], 'ro-', xnew, znew[0, :], 'b-')

#plt.show()

In [ ]:
vgrid.max()

In [ ]:
fig, ax = plt.subplots(1,3,figsize=(4,4))
#print(rhov.std(), vgrid.std())
#ax[0].imshow(rhov[2][:,:,0])
#ax[1].imshow(vgrid[2][:,:,0])
from meshoid import GridSurfaceDensity, GridSurfaceDensityMultigrid
%timeit sigma = GridSurfaceDensity(m, x, h, np.ones(3)*F["Header"].attrs["BoxSize"]/2,F["Header"].attrs["BoxSize"]*0.1, res=2048, box_size=-1,parallel=True)
ax[0].imshow(np.log10(sigma))
%timeit sigma2 = GridSurfaceDensityMultigrid(m, x, h, np.ones(3)*F["Header"].attrs["BoxSize"]/2,F["Header"].attrs["BoxSize"]*0.1, res=2048, box_size=-1,parallel=True, N_grid_kernel=8)
ax[1].imshow(np.log10(sigma2))
ax[2].imshow(np.log10(sigma2/sigma))
print(np.log10(sigma2/sigma).std())
#ax[0].imshow(rho.sum(axis=2))ne
#ax[1].imshow(np.log10(M.SurfaceDensity(m,res=256)))
#ax[1].imshow(vgrid2[0][:,:,0])
#ax[1].imshow(np.log10(M.Slice(density)))

In [ ]:
#vgrid = rhov
vk = np.array([fftn(V) for V in vgrid])
vkSqr = np.sum(np.abs(vk*vk),axis=0)

freqs = fftfreq(res)
print(freqs)
freq3d = np.array(np.meshgrid(freqs,freqs,freqs,indexing='ij'))
intfreq = np.int_(np.around(freq3d*res)+0.5)
kSqr = np.sum(np.abs(freq3d)**2,axis=0)
intkSqr = np.sum(np.abs(intfreq)**2, axis=0)
intk = intkSqr**0.5

#intkSqr_u = np.unique(intkSqr)
#print(intkSqr_u[:10])
kbins = np.arange(100) #np.unique(intkSqr**0.5)
#kbins = []; for k in np.sort(intkSqr)[:4]: kbins.append()
#kbins = np.logspace(-0.1,3,11)  #np.sort(np.unique(intk))[:1000]*0.99
#kbins = kbins + np.eps
#kbins = np.insert(kbins,0,0)
#kbins[1:] *= (1+1e-15)
#kbins[-1] = np.inf
from scipy.stats import binned_statistic

power_in_bin = binned_statistic(intk.flatten(), vkSqr.flatten(), bins=kbins,statistic='sum')[0]
power_spectrum = power_in_bin / np.diff(kbins) # power density in k space
power_spectrum *= np.sum(0.5 * m[:,None] * (v/1e3)**2) / power_spectrum.sum()
power_spectrum[power_spectrum == 0] = np.nan
#k_avg = (kbins[1:)
plt.plot(kbins[1:] , kbins[1:]**2 * power_spectrum,markersize=0.1); plt.yscale('log'); plt.xscale('log')
plt.xlabel("k")

plt.ylabel("$k^2 P(k)$")
#vavg = M.ProjectedAverage(v[:,2])

In [ ]:
print(np.unique(intkSqr**0.5)[:100])

In [ ]:
vk = np.array([fftn(V) for V in rhov])
vkSqr = np.sum(np.abs(vk*vk),axis=0)

freqs = fftfreq(res)
freq3d = np.array(np.meshgrid(freqs,freqs,freqs))#,indexing='ij'))
intfreq = np.around(freq3d*res)
kSqr = np.sum(np.abs(freq3d)**2,axis=0)
intkSqr = np.sum(np.abs(intfreq)**2, axis=0)
intk = intkSqr**0.5

kbins = np.logspace(-0.1,3,51)  #np.sort(np.unique(intk))[:1000]*0.99
#kbins = kbins + np.eps
#kbins = np.insert(kbins,0,0)
#kbins[1:] *= (1+1e-15)
#kbins[-1] = np.inf
from scipy.stats import binned_statistic


plt.plot(kbins[1:] , kbins[1:]**2 * binned_statistic(intk.flatten(), vkSqr.flatten(), bins=kbins,statistic='sum')[0],markersize=0.1); plt.yscale('log'); plt.xscale('log')
plt.xlabel("k")

plt.ylabel("$P(k) k^2$")
#vavg = M.ProjectedAverage(v[:,2])

In [ ]:
np.float64.small

In [ ]:
print(len(np.unique(intk)))

In [ ]:
rhov - vgrid